In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import VectorDBQA
import sentence_transformers
import pandas as pd

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 14653672
      0 drwxr-xr-x   56 kariato  staff   1.8K Apr  6 16:40 ./
      0 drwxr-xr-x   12 kariato  staff   384B Apr 18 01:58 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Apr  1 12:04 .DS_Store
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 21 00:04 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final (1).pdf
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 20 10:15 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final.pdf
     16 -rw-r--r--    1 kariato  staff   4.5K Apr  5 17:56 README.md
   3104 -rw-r--r--@   1 kariato  staff   1.5M Apr  7 19:12 alltopics.csv
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
      0 drwxr-xr-x    4 kariato  staff   128B Mar  5 11:36 chroma2_db/
      0 drwxr-xr-x    4 kariato  staff   128B Mar  5 11:36 chroma3_db/
      0 drwxr-xr-x    6 kariato  staff   192B Mar 18 00:30 chroma_db/
2785400 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
209969

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function2 = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

In [4]:
db = Chroma(embedding_function=embedding_function,persist_directory="./chroma_db")
db2 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma2_db")
db3 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma3_db")
#qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectordb)

In [5]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi3",temperature=0)

#x=llm.invoke(prompt)
#print(x)

In [7]:
import re
def cleanText(text):
        
    text = text.replace('\\n','')
    text = text.replace('\\','')
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

In [9]:
import time
from collections import defaultdict
scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query = "mark to market accounting practices"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral', 'llama3','llama3-chatqa','llama3-gradient','wizardlm2','phi3']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if email not in scores or model not in scores[email]:
        
            start_time = time.time()
            x=llm.invoke("""<s>[INST] <<sys>>
                         You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud. you output as a json text extractorusing the following format
                         Example of such a email in json format:
{"email_text":"I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud"},
{"reason":"The email shows concern that the accounting practices could be viewed as fraud"},{"Score":10},
{"email_text":"Accounting practices  did not report losses so revenue was realized as profit hidding true financial position"},
{"reason":"The email shows concern that the accounting practices are hiding fraud"},{"Score":10},
{"email_text":"Accounting practices  we placed on the books are 100% legal and trasparent"},
{"reason":"The email shows no concern, the accounting practices are legal and transparent"},{"Score":0},
{"email_text":"News about the accounting practices at Enron discuss devrivatives and mark to market accounting"},
{"Score":0},{"reason":"The email containing news reports not an action to commit fraud or hide loses"},

Please score the following email:
{"email_text":" """+email+'"}')
            scores[email][model]=x
            i+=1
            print("--- %s seconds ---" % (time.time() - start_time),i)
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix3_{model}.csv")

Querying db
600
256
Querying llm
0
Querying llm mistral
--- 19.68061900138855 seconds --- 1
--- 16.048987865447998 seconds --- 2
--- 2.827368974685669 seconds --- 3
--- 7.585156202316284 seconds --- 4
--- 18.226072788238525 seconds --- 5
--- 39.9639790058136 seconds --- 6
--- 9.422394037246704 seconds --- 7
--- 3.0571651458740234 seconds --- 8


In [29]:
results=[]
for i,j in scores.items():
    j['email']=i
    j.update(topics[i])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv("resultsMix3.csv")


In [31]:
resultsDF.columns


Index(['llama2-uncensored', 'email', 'DATE', 'SUBJECT', 'Unnamed: 0', 'score',
       'FROM', 'TO', 'SENT', 'CC'],
      dtype='object')

In [33]:
resultsDF.head().T

,0,1,2,3,4
mistral,<Score>0<End Score>\n\nThe email does not con...,<Score>0<End Score>\n\nThe email does not con...,<Score>0<End Score>\n\nThe email itself does ...,<Score>0<End Score>\n\nThe email does not con...,<Score>0<End Score>\n\nThe email does not con...
openhermes,<Score>0<End Score>,<Score>0<End Score>,<Score>0<End Score>,<Score>2<End Score>,<Score>0<End Score>
vicuna,<Score>8<End Score>,<Email Start>\nSubject: Daily Position Report ...,<Email Start>\nSubject: Re: WSJ Article Mark t...,<Email Start>Subject: WSJ Article on Enron's M...,<Score>7<End Score>
gemma,## Email Score\n\n**Email Start:**\n\nSubject:...,## Email Score\n\n**Email Start:**\n\nSubject:...,## Email Score\n\n**Email Start:**\n\nSubject:...,## Email Score\n\n**Email Start:**\n\nSubject:...,## Score for Email\n\n**Score:** 2\n\n**Explan...
llama2-uncensored,The email that discusses the news about Enron'...,The email with a score of 10 indicates an outr...,The first email is a 10/10 for abuse of accoun...,The email that discusses the mark to market ac...,"The email with the subject ""Roger Ondreko's Pr..."
email,Subject: Re: WSJ Article on Enron's Mark to Ma...,Subject: Daily Position Report class - sounds ...,Subject: WSJ Article Mark to Market MemoPlease...,Subject: WSJ Article on Enron's Mark to Market...,Subject: Roger Ondreko's Presentation on Marke...
DATE,"Fri, 22 Sep 2000 15:37","Wed, 23 Feb 2000 01:44","Wed, 20 Sep 2000 02:26","Fri, 22 Sep 2000 10:33","Tue, 24 Oct 2000 05:25"
SUBJECT,Re,>>,NaN,NaN,>>
Unnamed: 0,77277,384559,227836,59549,301715
score,0.562741,0.703303,0.758774,0.771171,0.780912


In [35]:
import time
from collections import defaultdict
scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query =  "raptor creation of special purpose entities"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if email not in scores or model not in scores[email]:
        
            start_time = time.time()
            x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
Example of such a email:
<Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
<Email End>
<Score>10<End Score>
<Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
<Email End>
<Score>10<End Score>                 
<Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
<Email End>
<Score>0<End Score>
<Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
<Email End>
<Score>0<End Score>                     
Please score the following email:
<Email Start>
{email}
<Email End>
""")
            scores[email][model]=x
            i+=1
            print("--- %s seconds ---" % (time.time() - start_time),i)
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix3_{model}.csv")

Querying db
600
290
Querying llm
0
Querying llm mistral
--- 17.531227827072144 seconds --- 1
--- 4.557079076766968 seconds --- 2
--- 3.0837929248809814 seconds --- 3
--- 3.118680000305176 seconds --- 4
--- 5.81890082359314 seconds --- 5
--- 3.908447027206421 seconds --- 6
--- 4.15209698677063 seconds --- 7
--- 4.503992795944214 seconds --- 8
--- 3.660292863845825 seconds --- 9
--- 5.819736957550049 seconds --- 10
--- 3.52354097366333 seconds --- 11
--- 3.8482720851898193 seconds --- 12
--- 2.6564719676971436 seconds --- 13
--- 2.655046224594116 seconds --- 14
--- 4.0458948612213135 seconds --- 15
--- 7.649841785430908 seconds --- 16
--- 2.274600028991699 seconds --- 17
--- 3.857692241668701 seconds --- 18
--- 3.540065050125122 seconds --- 19
--- 2.9082581996917725 seconds --- 20
--- 4.050575256347656 seconds --- 21
--- 3.956775188446045 seconds --- 22
--- 4.052804231643677 seconds --- 23
--- 4.076849937438965 seconds --- 24
--- 4.99104905128479 seconds --- 25
--- 5.135876893997192 seco

In [41]:
import time
from collections import defaultdict
#scores=defaultdict(dict)
skip=1
query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
#query =  "raptor creation of special purpose entities"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if (email not in scores or model not in scores[email]) :
            if skip==0:
                start_time = time.time()
                x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email}
    <Email End>
    """)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                skip-=1
                scores[email][model]='hang'
                print(f"Skipping {email} {model} {skip}")
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix4_{model}.csv")

Querying db
600
342
Querying llm
342
Querying llm mistral
Already scored 1
Already scored 2
Already scored 3
Already scored 4
Already scored 5
Already scored 6
Already scored 7
Already scored 8
Already scored 9
Already scored 10
Already scored 11
Already scored 12
Already scored 13
Already scored 14
Already scored 15
Already scored 16
Already scored 17
Already scored 18
Already scored 19
Already scored 20
Already scored 21
Already scored 22
Already scored 23
Already scored 24
Already scored 25
Already scored 26
Already scored 27
Already scored 28
Already scored 29
Already scored 30
Already scored 31
Already scored 32
Already scored 33
Already scored 34
Already scored 35
Already scored 36
Already scored 37
Already scored 38
Already scored 39
Already scored 40
Already scored 41
Already scored 42
Already scored 43
Already scored 44
Already scored 45
Already scored 46
Already scored 47
Already scored 48
Already scored 49
Already scored 50
Already scored 51
Already scored 52
Already scored 

In [42]:
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix4a_{model}.csv")
resultsDF.shape

(342, 14)

In [44]:
scores=defaultdict(dict)
skip=0
M8="mark to market losses"
M2= "mark to market accounting practices"
M6="record keeping accounting practices"
M4="it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
M3="raptor creation of special purpose entities"

In [45]:
import time
from collections import defaultdict
#scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query =  "record keeping accounting practices"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if (email not in scores or model not in scores[email]) :
            if skip==0:
                start_time = time.time()
                x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email}
    <Email End>
    """)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                skip-=1
                scores[email][model]='hang'
                print(f"Skipping {email} {model} {skip}")
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix6_{model}.csv")

Querying db
600
307
Querying llm
0
Querying llm mistral
--- 8.236916065216064 seconds --- 1
--- 6.371081113815308 seconds --- 2
--- 10.25141716003418 seconds --- 3
--- 5.6597819328308105 seconds --- 4
--- 8.338778972625732 seconds --- 5
--- 10.389668941497803 seconds --- 6
--- 3.352882146835327 seconds --- 7
--- 7.822736740112305 seconds --- 8
--- 8.365654945373535 seconds --- 9
--- 7.460535049438477 seconds --- 10
--- 14.644362926483154 seconds --- 11
--- 7.490654945373535 seconds --- 12
--- 3.614694833755493 seconds --- 13
--- 15.754616022109985 seconds --- 14
--- 8.373269081115723 seconds --- 15
--- 5.254549980163574 seconds --- 16
--- 6.979875087738037 seconds --- 17
--- 3.751911163330078 seconds --- 18
--- 5.771327018737793 seconds --- 19
--- 8.62169599533081 seconds --- 20
--- 6.756669044494629 seconds --- 21
--- 3.910100221633911 seconds --- 22
--- 6.971081256866455 seconds --- 23
--- 8.2864511013031 seconds --- 24
--- 8.029568910598755 seconds --- 25
--- 4.902186155319214 secon

In [46]:
scores=defaultdict(dict)
skip=0

In [49]:
skip=1


In [50]:
import time
query = "mark to market losses"
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if (email not in scores or model not in scores[email]) :
            if skip==0:
                start_time = time.time()
                x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email}
    <Email End>
    """)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                skip-=1
                scores[email][model]='hang'
                print(f"Skipping {email} {model} {skip}")
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix8_{model}.csv")

600
277
Querying llm
277
Querying llm mistral
Already scored 1
Already scored 2
Already scored 3
Already scored 4
Already scored 5
Already scored 6
Already scored 7
Already scored 8
Already scored 9
Already scored 10
Already scored 11
Already scored 12
Already scored 13
Already scored 14
Already scored 15
Already scored 16
Already scored 17
Already scored 18
Already scored 19
Already scored 20
Already scored 21
Already scored 22
Already scored 23
Already scored 24
Already scored 25
Already scored 26
Already scored 27
Already scored 28
Already scored 29
Already scored 30
Already scored 31
Already scored 32
Already scored 33
Already scored 34
Already scored 35
Already scored 36
Already scored 37
Already scored 38
Already scored 39
Already scored 40
Already scored 41
Already scored 42
Already scored 43
Already scored 44
Already scored 45
Already scored 46
Already scored 47
Already scored 48
Already scored 49
Already scored 50
Already scored 51
Already scored 52
Already scored 53
Already s

In [ ]:
import time
from collections import defaultdict
scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query = "mark to market accounting practices"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if email not in scores or model not in scores[email]:
        
            start_time = time.time()
            x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
Example of such a email:
<Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
<Email End>
<Score>10<End Score>
<Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
<Email End>
<Score>10<End Score>                 
<Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
<Email End>
<Score>0<End Score>
<Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
<Email End>
<Score>0<End Score>                     
Please score the following email:
<Email Start>
{email}
<Email End>
""")
            scores[email][model]=x
            i+=1
            print("--- %s seconds ---" % (time.time() - start_time),i)
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix2_{model}.csv")

Querying db
600
256
Querying llm
0
Querying llm mistral
--- 18.146378993988037 seconds --- 1
--- 3.9028940200805664 seconds --- 2
--- 3.522249937057495 seconds --- 3
--- 4.473380088806152 seconds --- 4
--- 7.698180198669434 seconds --- 5
--- 6.41375207901001 seconds --- 6
--- 6.836760997772217 seconds --- 7
--- 4.954282999038696 seconds --- 8
--- 7.413327693939209 seconds --- 9
--- 7.776710033416748 seconds --- 10
--- 7.828747034072876 seconds --- 11
--- 12.7291579246521 seconds --- 12
--- 4.249345064163208 seconds --- 13
--- 9.960734844207764 seconds --- 14
--- 4.316695213317871 seconds --- 15
--- 11.106553077697754 seconds --- 16
--- 4.659502983093262 seconds --- 17
--- 9.56882095336914 seconds --- 18
--- 12.109672784805298 seconds --- 19
--- 4.797725200653076 seconds --- 20
--- 3.3922412395477295 seconds --- 21
--- 2.8195438385009766 seconds --- 22
--- 10.160510063171387 seconds --- 23
--- 10.024769067764282 seconds --- 24
--- 6.506580829620361 seconds --- 25
--- 8.296093940734863 s

In [9]:
enron_df=pd.read_csv('enron_processed.csv')

In [10]:
print(enron_df.shape)
enron_df.drop_duplicates(subset='body_short',inplace=True) 
print(enron_df.shape)


(517401, 15)
(246415, 15)


In [11]:

enron_df=enron_df[enron_df['body_short'].str.len()>30]
enron_sampple_df= enron_df.sample(1000)
enron_data=enron_sampple_df['body_short'].tolist()

In [17]:
topics={}
for i,j in enron_sampple_df.iterrows():
    topics[j['body_short']]=j.to_dict()

In [22]:
from collections import defaultdict
def split_string(string, chunk_size):
    return [string[i:i+chunk_size] for i in range(0, len(string), chunk_size)]
scores=defaultdict(dict)
skip=0

In [24]:
skip=1

In [27]:
import time
print(len(enron_data))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    ii=0
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email_long in enron_data:
        for email in split_string(email_long,2500):
            topics[email]=topics[email_long]
            if (email not in scores or model not in scores[email]) :
                if skip==0:
                    start_time = time.time()
                    
                    if (email not in scores or model not in scores[email]) :
                        if skip==0:
                            start_time = time.time()
                            x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email.replace('http://',' ')}
    <Email End>
    """)
                        scores[email][model]=x
                        i+=1
                        ii+=1
                        print("--- %s seconds ---" % (time.time() - start_time),i,model,ii)
                    else:
                        skip-=1
                        scores[email][model]='hang'
                        print(f"Skipping {email} {model} {skip}")
                else:
                    i+=1
                    print(f"Already scored {i}")
                #print(x)
        #print(x)
            # if i%20000==0:
            #     results=[]
            #     for idx,j in scores.items():
            #         j['email']=idx
            #         j.update(topics[idx])
            #         results.append(j)
            #     resultsDF=pd.DataFrame(results)
            #     resultsDF.to_csv(f"resultsMixBlank_{model}.csv")
results=[]
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMixBlank_{model}.csv")

1000
Querying llm mistral
Querying llm openhermes
Querying llm vicuna
Querying llm gemma
Querying llm llama2-uncensored
Already scored 1
Already scored 2
Already scored 3
Already scored 4
Already scored 5
Already scored 6
Already scored 7
Already scored 8
Already scored 9
Already scored 10
Already scored 11
Already scored 12
Already scored 13
Already scored 14
Already scored 15
Already scored 16
Already scored 17
Already scored 18
Already scored 19
Already scored 20
Already scored 21
Already scored 22
Already scored 23
Already scored 24
Already scored 25
Already scored 26
Already scored 27
Already scored 28
Already scored 29
Already scored 30
Already scored 31
Already scored 32
Already scored 33
Already scored 34
Already scored 35
Already scored 36
Already scored 37
Already scored 38
Already scored 39
Already scored 40
Already scored 41
Already scored 42
Already scored 43
Already scored 44
Already scored 45
Already scored 46
Already scored 47
Already scored 48
Already scored 49
Already

In [149]:
import re
def score(result):
        y=None
        if '<Score>' in result:
            if '<End Score>' in result:
                y=re.findall(r'<Score>(.*?)<End Score>', result)[0]
            elif '</Score>' in result:
                y=re.findall(r'<Score>(.*?)</Score>', result)[-1]
        elif '**Score:**' in result:
            y=re.findall(r'\*\*Score:\*\* \d+',result)[0][10:]
        elif '**Email Score:** ' in result:
            y=re.findall(r'\*Email Score:\*\* \d+',result)[0][16:]
        elif 'a score of ' in result:
            y=re.findall(r'a score of (.*?)',result)
        elif 'The email score is ' in result:
            y=re.findall(r'The email score is \d+',result)[10:]
        elif 'Score: ' in result:
            y=re.findall(r'Score: \d+',result)[-1][7:]
        elif '## Score for the email:' in result:
            numbers=re.findall(r'\d+', result)
            y=numbers[0]
        elif 'I would score this email as ' in result:
            numbers=re.findall(r'I would score this email as \d+', result)
            if len(numbers)==0:
                y=0
            else:
                y=numbers[0][28]
        else:    
            numbers=re.findall(r'\d+', result)
            if len(numbers)==0:
                y=0
            elif len(numbers)==1:
                y=numbers[0]
            else:               
                y=""
        if y is not None and isinstance(y,list):
            if len(y)==0:
                y=''
            else:
                y=y[0]
        if y=='':
            y=0
        try:
            y = int(y)
        except ValueError:
            print(y)
            y=0
        except TypeError:
            y=0
        return y
resultsDF.columns
models=['mistral','openhermes','vicuna', 'gemma']
for model in models:
    resultsDF[f'{model}_score']=resultsDF[model].apply(score)
models2=[i+'_score' for i in models]
resultsDF[models2]
    

,mistral_score,openhermes_score,vicuna_score,gemma_score
0,0,0,0,0
1,0,0,7,6
2,0,0,5,0
3,0,0,7,0
4,0,0,0,0
...,...,...,...,...
1325,0,0,5,2
1326,0,0,0,2
1327,0,0,8,0
1328,0,0,0,0


In [130]:
resultsDF['mistral_score'].value_counts()

mistral_score
0     1295
5       18
10      11
8        2
7        2
3        1
2        1
Name: count, dtype: int64

In [131]:
resultsDF['openhermes_score'].value_counts()

openhermes_score
0     1271
2       19
10      15
5       13
3        4
1        3
8        2
25       1
6        1
4        1
Name: count, dtype: int64

In [132]:
resultsDF['vicuna_score'].value_counts()

vicuna_score
7      443
0      304
8      220
5      199
9      104
6       24
3       16
2        7
10       7
4        3
1        1
100      1
13       1
Name: count, dtype: int64

In [133]:
resultsDF['gemma_score'].value_counts()

gemma_score
0     708
2     422
6     125
8      48
10     17
3       8
4       2
Name: count, dtype: int64

In [138]:
resultsDF[resultsDF['mistral_score']==10]['email']

317     Subject: Enron Mentions -- 02/04/02\n\n\nEx-En...
321     rence to say they will vote Tuesday to subpoen...
326     ," Silbert wrote.\n       "These inflammatory ...
327     the Sunday political talk shows. Lawmakers dre...
328     s Meet the Press that his own committee's inve...
329      11:31 (New York)\n     Washington, Feb. 4 (Bl...
330     closures in Enron's public filings, along with...
331     gs restatements, summarised his findings by sa...
332     people," in and outside the company, including...
336     dollar mansion.\n\nAP Photos HT102-104 \nCopyr...
1047    Subject: RE: California Rate Exposure\n\nHere ...
Name: email, dtype: object

In [ ]:
resultsDF[resultsDF['gemma_score']==10]['email']

149      you were crying and everyone around you was \...
234                                     - RULE14 6-23.doc
393      week,\nhad never been in any trouble before a...
481     en/topics/segtopic_eep_sub_ctr.htm?DGVCode%3DE...
681      free to call us, should you need any clarific...
700     ity customers by mid-year.  Call\nIntercept bl...
780     l Member Bell is an attorney with Beirne, Mayn...
788     lients, a telecommunication=\ns company, to ch...
853     ee of these\n\n(This week's guest wordsmith, R...
872     formation to me, the number is 505/855-6240.\n...
906      remember that!\n>\n> And was it really that l...
928     xhibit GG in the TSIA, LOH is defined in short...
945     iser recommends a bubbly such as the 1990=20\n...
1055    tercepted by a third party.  For your protecti...
1119    tely settle on for the longer serving employee...
1168    ton, Texas 77002\nDirect Tel:  713.650.2737\nC...
1283    mmunication skills\n? Working knowledge of Exc...
Name: email, d

In [166]:
results2_df=pd.read_csv('resultsMix2_all.csv')
results3_df=pd.read_csv('resultsMix3_all.csv')
results4_df=pd.read_csv('resultsMix4a_all.csv')
results6_df=pd.read_csv('resultsMix6_all.csv')
results8_df=pd.read_csv('resultsMix8_all.csv')
resultsB_df=pd.read_csv('resultsMixBlankG_all.csv')
results8_df['search_phrase']="mark to market losses"
results2_df['search_phrase']= "mark to market accounting practices"
results6_df['search_phrase']="record keeping accounting practices"
results4_df['search_phrase']="it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
results3_df['search_phrase']="raptor creation of special purpose entities"
resultsB_df['search_phrase']='RANDOM'


In [167]:
models=['mistral','openhermes','vicuna', 'gemma']
for model in models:
    results2_df[f'{model}_score']=results2_df[model].apply(score)
    results3_df[f'{model}_score']=results3_df[model].apply(score)
    results4_df[f'{model}_score']=results4_df[model].apply(score)
    results6_df[f'{model}_score']=results6_df[model].apply(score)
    results8_df[f'{model}_score']=results8_df[model].apply(score)
    resultsB_df[f'{model}_score']=resultsB_df[model].apply(score)
models2=[i+'_score' for i in models]

0</Score>
7/10
4/10
7/10


In [171]:
results_merged_df=pd.concat([results2_df,results3_df,results4_df,results6_df,results8_df,resultsB_df])
results_merged_df.head(1).T

,0
Unnamed: 0.1,0
mistral,<Score>0<End Score>\n\nThe email does not con...
openhermes,<Score>0<End Score>
vicuna,<Score>8<End Score>
gemma,## Email Score\n\n**Email Start:**\n\nSubject:...
llama2-uncensored,The email that discusses the news about Enron'...
email,Subject: Re: WSJ Article on Enron's Mark to Ma...
DATE,"Fri, 22 Sep 2000 15:37"
SUBJECT,Re
Unnamed: 0,77277


In [179]:
# Assuming you have a DataFrame called df and the column you want to check for duplicates is 'column_name'
duplicates = results_merged_df[results_merged_df['Unnamed: 0'].duplicated()]
duplicates.head()
results_merged_df.to_csv('results_merged_df.csv')



In [178]:
tmp=results_merged_df[results_merged_df['Unnamed: 0']==114797]
tmp['search_phrase']

22     mark to market accounting practices
25     mark to market accounting practices
194                  mark to market losses
Name: search_phrase, dtype: object

In [151]:
results2_df['mistral_score'].value_counts()

mistral_score
0     234
5      10
10      6
3       4
7       1
8       1
Name: count, dtype: int64

In [152]:
results3_df['mistral_score'].value_counts()

mistral_score
0    266
5     16
3      5
8      2
7      1
Name: count, dtype: int64

In [153]:
results4_df['mistral_score'].value_counts()

mistral_score
0     218
10     54
5      32
8      24
7       5
3       3
9       3
6       3
Name: count, dtype: int64

In [165]:
results8_df[results8_df['mistral_score']==8]['email']

174    -losing investments but to hide losses in its ...
238    ng more disclosures about the accounting used,...
Name: email, dtype: object

In [154]:
results6_df['mistral_score'].value_counts()

mistral_score
0     258
10     16
5      13
3      12
8       6
7       2
Name: count, dtype: int64

In [156]:
results8_df.columns

Index(['Unnamed: 0.1', 'mistral', 'openhermes', 'vicuna', 'gemma',
       'llama2-uncensored', 'email', 'DATE', 'Unnamed: 0', 'score', 'SUBJECT',
       'TO', 'FROM', 'SENT', 'CC', 'mistral_score', 'openhermes_score',
       'vicuna_score', 'gemma_score'],
      dtype='object')

In [155]:
results8_df['mistral_score'].value_counts()

mistral_score
0     256
3       7
5       6
10      3
7       2
8       2
2       1
Name: count, dtype: int64

In [198]:
results_merged_df[['mistral_score', 'openhermes_score','vicuna_score', 'gemma_score','search_phrase','email','Unnamed: 0']].to_excel('results_merged_df.xlsx')
#.groupby('search_phrase').count()

In [195]:
results_merged_df[results_merged_df['mistral_score']>9]['email']

13      Subject: The first detailed allegations that E...
15      ancial reporting technique called mark-to-mark...
32      "It is basic accounting that you don't record ...
34      derwrite the project, if it ever would. But be...
54      Bad trading. Enron's main technique to pump up...
                              ...                        
330     closures in Enron's public filings, along with...
331     gs restatements, summarised his findings by sa...
332     people," in and outside the company, including...
336     dollar mansion.\n\nAP Photos HT102-104 \nCopyr...
1047    Subject: RE: California Rate Exposure\n\nHere ...
Name: email, Length: 90, dtype: object